# Reconhecedor de Placas de Sinalização de Trânsito
### PSI3571 - Práticas em Reconhecimento de Padrões, Modelagem e Inteligência Computacional

Atividade da P2 da disciplina PSI3571

## Setup

In [ ]:
import numpy as np
import pandas as pd
import math

import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from skimage import exposure
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

%matplotlib inline

In [ ]:
random_state = 3571

np.random.seed(random_state)

In [ ]:
# link = 'GTSRB/Final_Training/Images/' # Local
link = '../input/gtsrb-german-traffic-sign/' # Kaggle

## Descrição

Observando a descrição do dataset, é possível encontrar a relação entre os valores numéricos de ```ClassId``` e as descrições das placas com classificação e tipo de cada placa.

A informação de tipo é importante pois placas de mesmo tipo têm forma similar. Por exemplo, placas do tipo ```prohibitory``` são circulares e com borda vermelha, enquanto placas do tipo ```danger``` são triangulares e também com borda vermelha. Já as placas ```mandatory``` são placas circulares e preenchidas em azul. Por fim, as placas identificada como ```other``` possuem formas distintas.

In [ ]:
GTSRBInfo = pd.DataFrame({
    0:  { 'Type': 'prohibitory',  'Label': "speed limit 20"                         },
    1:  { 'Type': 'prohibitory',  'Label': "speed limit 30"                         },
    2:  { 'Type': 'prohibitory',  'Label': "speed limit 50"                         },
    3:  { 'Type': 'prohibitory',  'Label': "speed limit 60"                         },
    4:  { 'Type': 'prohibitory',  'Label': "speed limit 70"                         },
    5:  { 'Type': 'prohibitory',  'Label': "speed limit 80"                         },
    6:  { 'Type': 'other',        'Label': "restriction ends 80"                    },
    7:  { 'Type': 'prohibitory',  'Label': "speed limit 100"                        },
    8:  { 'Type': 'prohibitory',  'Label': "speed limit 120"                        },
    9:  { 'Type': 'prohibitory',  'Label': "no overtaking"                          },
    10: { 'Type': 'prohibitory',  'Label': "no overtaking (trucks)"                 },
    11: { 'Type': 'danger',       'Label': "priority at next intersection"          },
    12: { 'Type': 'other',        'Label': "priority road"                          },
    13: { 'Type': 'other',        'Label': "give way"                               },
    14: { 'Type': 'other',        'Label': "stop"                                   },
    15: { 'Type': 'prohibitory',  'Label': "no traffic both ways"                   },
    16: { 'Type': 'prohibitory',  'Label': "no trucks"                              },
    17: { 'Type': 'other',        'Label': "no entry"                               },
    18: { 'Type': 'danger',       'Label': "danger"                                 },
    19: { 'Type': 'danger',       'Label': "bend left"                              },
    20: { 'Type': 'danger',       'Label': "bend right"                             },
    21: { 'Type': 'danger',       'Label': "bend"                                   },
    22: { 'Type': 'danger',       'Label': "uneven road"                            },
    23: { 'Type': 'danger',       'Label': "slippery road"                          },
    24: { 'Type': 'danger',       'Label': "road narrows"                           },
    25: { 'Type': 'danger',       'Label': "construction"                           },
    26: { 'Type': 'danger',       'Label': "traffic signal"                         },
    27: { 'Type': 'danger',       'Label': "pedestrian crossing"                    },
    28: { 'Type': 'danger',       'Label': "school crossing"                        },
    29: { 'Type': 'danger',       'Label': "cycles crossing"                        },
    30: { 'Type': 'danger',       'Label': "snow"                                   },
    31: { 'Type': 'danger',       'Label': "animals"                                },
    32: { 'Type': 'other',        'Label': "restriction ends"                       },
    33: { 'Type': 'mandatory',    'Label': "go right"                               },
    34: { 'Type': 'mandatory',    'Label': "go left"                                },
    35: { 'Type': 'mandatory',    'Label': "go straight"                            },
    36: { 'Type': 'mandatory',    'Label': "go right or straight"                   },
    37: { 'Type': 'mandatory',    'Label': "go left or straight"                    },
    38: { 'Type': 'mandatory',    'Label': "keep right"                             },
    39: { 'Type': 'mandatory',    'Label': "keep left"                              },
    40: { 'Type': 'mandatory',    'Label': "roundabout"                             },
    41: { 'Type': 'other',        'Label': "restriction ends (overtaking)"          },
    42: { 'Type': 'other',        'Label': "restriction ends (overtaking (trucks))" },
}).T

# Adiciona ClassId como coluna da tabela
GTSRBInfo.index.name = 'ClassId'
GTSRBInfo.reset_index(inplace=True)

# Transforma Label em valor categórico
GTSRBInfo['Label'] = GTSRBInfo.Label.astype('category')

# Transforma Type em valor categórico e ordena de acordo com a quantidade de dados
Types = GTSRBInfo.Type.value_counts().index
GTSRBInfo['Type'] = GTSRBInfo.Type.astype('category').cat.reorder_categories(Types)
GTSRBInfo['TypeId'] = GTSRBInfo.Type.cat.codes

GTSRBInfo

In [ ]:
def plotTrafficSigns(dataset, imRead=lambda classe: plt.imread(classe.Path), maxCols=5, random_state=None):
    for Type in Types:
        classes = dataset[dataset.Type == Type]
        classLabels = classes.ClassId.unique()

        nRows = math.ceil(classLabels.size / maxCols)
        nCols = min(classLabels.size, maxCols)

        fig, axs = plt.subplots(
            nrows = nRows,
            ncols = nCols,
            figsize = (4*nCols, 3*nRows + 2),
        )

        for i, pos in enumerate(np.ndindex(axs.shape)):
            try:
                classe = dataset[dataset.ClassId == classLabels[i]].sample(1, random_state=random_state).iloc[0]

                axs[pos].imshow(imRead(classe))
                axs[pos].set_title("{}: {}".format(classe.ClassId, classe.Label))

            except:
                pass

            axs[pos].axis('off')

        fig.suptitle(Type.capitalize(), fontsize='xx-large')
        plt.show()

In [ ]:
# Apenas no kaggle
metaRead = lambda classe: plt.imread("../input/gtsrb-german-traffic-sign/Meta/{}.png".format(classe.ClassId))
                                     
plotTrafficSigns(dataset=GTSRBInfo, imRead=metaRead)

In [ ]:
sns.countplot(
    x='Type',
    data=GTSRBInfo,
).set_title('Quantidade de placas de cada formato')

plt.show()

## Carregando o dataset

Vamos primeiramente carregar o dataset.

In [ ]:
%%time

# # Concatena os diferentes .csv em um único DataFrame (Apenas local)
# fileName = lambda i: link + "{0:05d}/GT-{0:05d}.csv".format(int(i))
# GTSRB = pd.concat((pd.read_csv(fileName(i), delimiter = ';') for i in GTSRBInfo.ClassId), ignore_index = True)

# # Corrige o caminho das figuras
# GTSRB.Filename = GTSRB.ClassId.map(lambda ID: link + "{:05d}/".format(ID)) + GTSRB.Filename

# Carrega o .csv único disponível (Apenas no Kaggle)
GTSRB = pd.read_csv(link + "Train.csv")
GTSRB.Path = link + GTSRB.Path

# União do dataset e das descrições
GTSRB = GTSRB.merge(GTSRBInfo, on="ClassId")

# Cria lista ordenada das placas disponíveis
Labels = GTSRB.Label.value_counts().index

GTSRB.shape

In [ ]:
GTSRB.sample(5)

## Análise dos dados

### Informações sobre as figuras

Inicialmente, vamos observar dados relativos aos valores de ```Height``` e ```Width``` das figuras disponíveis. Aqui pode-se notar que as figuras não são necessariamente quadradas e que possuem tamanhos distintos, variando, em cada eixo, de 25 pixels a até mais de 200 pixels.

In [ ]:
GTSRB[['Width', 'Height']].describe()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharex=True)

sns.distplot(
    GTSRB.Width,
    bins=25,
    kde=False,
    color='red',
    ax=axs[0],
)
axs[0].set_title("Distribuição de valores de largura (em px)")
axs[0].set_xlabel("Largura")
axs[0].set_ylabel("Frequência")

sns.distplot(
    GTSRB.Height,
    bins=25,
    kde=False,
    color='blue',
    ax=axs[1]
)
axs[1].set_title("Distribuição de valores de altura (em px)")
axs[1].set_xlabel("Altura")
axs[1].set_ylabel("Frequência")

plt.show()

Outra informação relacionada é a largura e a altura efetivamente úteis em cada imagem:  

$$H_{efet} = |Y_2 - Y_1|$$
$$W_{efet} = |X_2 - X_1|$$

In [ ]:
H_efet = (GTSRB['Roi.Y2'] - GTSRB['Roi.Y1']).abs()
W_efet = (GTSRB['Roi.X2'] - GTSRB['Roi.X1']).abs()

W_efet.name = 'Largura efetiva'
H_efet.name = 'Altura efetiva'

pd.concat([W_efet, H_efet], axis=1, names=['a', 'b']).describe()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5), sharex=True)

sns.distplot(
    W_efet,
    bins=25,
    kde=False,
    color='red',
    ax=axs[0]
)
axs[0].set_title("Distribuição de valores de largura útil (em px)")
axs[0].set_ylabel("Frequência")

sns.distplot(
    H_efet,
    bins=25,
    kde=False,
    color='blue',
    ax=axs[1]
)
axs[1].set_title("Distribuição de valores de altura útil (em px)")
axs[1].set_ylabel("Frequência")

plt.show()

### Informações sobre as placas disponíveis

Outra informação relevante se dá sobre a quantidade de dados disponíveis para cada placa:

In [ ]:
sns.countplot(
    x='Type',
    data=GTSRB,
).set_title("Quantidade de exemplos para cada tipo de placa")

plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.xticks(rotation=90)

LabelsType = GTSRBInfo.set_index('Label').loc[Labels].TypeId
palette = np.asarray(sns.color_palette())[LabelsType]

sns.countplot(
    x='Label',
    data=GTSRB,
    palette=palette,
    order=Labels,
).set_title("Quantidades de exemplos para cada placa")

plt.legend(handles=[
    mpatches.Patch(color=barColor, label=barType)
    for barColor, barType
    in zip(sns.color_palette(), Types)
])

plt.show()

### Visualização das placas

Vamos agora observar um exemplo de cada placa presente no dataset.

In [ ]:
plotTrafficSigns(GTSRB, random_state=random_state)

## Tratamento dos dados

Agora que já pudemos observar os dados disponíveis, faremos alguns tratamentos para possibilitar o uso das imagens pelo modelo e para obtermos melhores resultados.

### Data Augmentation

Como nossos conjunto de dados não possui tantas imagens, uma prática recomendada nos próprios [exemplos do keras](https://keras.io/examples/vision/image_classification_from_scratch/) é a de usar a técnica de *data augmentation*. Através dela, podemos gerar novas imagens a partir das imagens presentes no dataset através da aplicação de pequenas transformações, como por exemplos pequenas rotações ou translações da imagem original, conforme definiremos abaixo.

In [ ]:
dataAugmentation = {
    'rotation_range': 10,
    'zoom_range': 0.15,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'shear_range': 0.15,
}

imageGenerator = ImageDataGenerator(**dataAugmentation)

### Tratamento e leitura das imagens

Neste item definiremos uma série de tratamentos sobre cada imagem que será carregada, para que elas possam ser corretamente mapeadas para as entradas da rede neural e para que os resultados do modelo sejam melhores. Os tratamentos feitos serão:

1. Redimensionar imagens para um mesmo tamanho  
   O objetivo aqui é que todas as imagens tenham o mesmo número de valores para que eles possam ser mapeados na entrada da rede neural.

1. Normalizar o contraste nas figuras  
   O objetivo é tornar os detalhes das placas mais nítidos e mais distintos de elementos de fundo.

1. A região de interesse, onde de fato a placa se encontra, não necessariamente equivale a toda a área da imagem. (TODO)

In [ ]:
imageShape = {
    'height': 32,
    'width': 32,
    'depth': 3,
}

def imageTreatment(image):
    treatedImage = cv2.resize(image, (imageShape['width'], imageShape['height']))
    treatedImage = exposure.equalize_adapthist(treatedImage, clip_limit=0.1)
    # Corte?

    return treatedImage

In [ ]:
%%time

Images = GTSRB.Path.map(plt.imread).map(imageTreatment)

#### Visualização dos resultados do tratamento

In [ ]:
plotTrafficSigns(GTSRB, imRead=lambda classe: Images[classe.name], random_state=random_state)

### Codificação das classes de saída

Nesta etapa faremos um procedimento adicional para que os dados possam ser utilizados pelo nosso modelo de rede neural:
   
1. Aplicação de One-Hot Encoding nos valores de `Label`, usados como saída do modelo
   Este tratamento é necessário pois a rede neural só é capaz de fornecer resultados de ponto flutuante em um pequeno intervalo e estes valores estão associados à intensidade da ativação de um determinado neurônio. Assim, uma maneira de obter um resultado melhor é utilizando a técnica de One-Hot Encoding e realizar a classificação com base no neurônio de saída que ficou ativo com maior intensidade.

In [ ]:
oneHotLabels = pd.get_dummies(GTSRB.Label)

### Divisão em dados de treinamento e de validação

Ainda antes de iniciarmos o treinamento, vamos dividir o dataset em um conjunto de dados especifiamente para treinamento e outro conjunto de dados para validação dos resultados. Assim, poderemos observar a qualidade do classificador com base em métricas de interesse e com dados que não foram utilizados anteriormente.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    np.stack(Images),
    oneHotLabels,
    test_size=0.10,
    random_state=random_state
)

print("Dados de treinamento: {:5d}/{}".format(len(X_train), len(GTSRB)))
print("Dados de validação:   {:5d}/{}".format(len(X_val), len(GTSRB)))

## Treinamento - Classificação

Agora que já temos todos os dados carregadores e que já observamos as informações que temos disponíveis, vamos iniciar o treinamento de um modelo para a classificação de placas de trânsito, ou seja, dada uma imagem de uma placa, o modelo deve nos fornecer o correto valor de `ClassId`.

### Definição do modelo

Vamos primeiramente criar um modelo baseado em Redes Neurais utilizando o `Keras`. Nesta etapa será criada a **Rede Neural** e definiremos quais serão as camadas desta rede.

In [ ]:
nClasses = Labels.size
inputShape = (imageShape['height'], imageShape['width'], imageShape['depth'])

TrafficSignNet = Sequential()

# Primeira camada: Convolucional -> BatchNormalization -> MaxPooling
TrafficSignNet.add(Conv2D(8, (5, 5), padding="same", input_shape=inputShape))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization(axis=-1))
TrafficSignNet.add(MaxPooling2D(pool_size=(2, 2)))

# Segunda camada: Convolucional -> Relu -> BatchNormalization -> Convolucional -> Relu -> BatchNormalization -> MaxPooling
TrafficSignNet.add(Conv2D(16, (3, 3), padding="same"))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization(axis=-1))
TrafficSignNet.add(Conv2D(16, (3, 3), padding="same"))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization(axis=-1))
TrafficSignNet.add(MaxPooling2D(pool_size=(2, 2)))

# Terceira camada: Convolucional -> Relu -> BatchNormalization -> Convolucional -> Relu -> BatchNormalization -> MaxPooling
TrafficSignNet.add(Conv2D(32, (3, 3), padding="same"))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization(axis=-1))
TrafficSignNet.add(Conv2D(32, (3, 3), padding="same"))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization(axis=-1))
TrafficSignNet.add(MaxPooling2D(pool_size=(2, 2)))

# Quarta camada: Flatten -> Relu -> BatchNormalization -> DropOut
TrafficSignNet.add(Flatten())
TrafficSignNet.add(Dense(128))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization())
TrafficSignNet.add(Dropout(0.5))

# Quinta camada: Flatten -> Relu -> BatchNormalization -> DropOut
TrafficSignNet.add(Flatten())
TrafficSignNet.add(Dense(128))
TrafficSignNet.add(Activation("relu"))
TrafficSignNet.add(BatchNormalization())
TrafficSignNet.add(Dropout(0.5))

# Sexta camada: Softmax
TrafficSignNet.add(Dense(nClasses))
TrafficSignNet.add(Activation("softmax"))

Por fim, vamos definir os pesos de acordo com o número de elementos de cada classe. Esta abordagem é usada por conta do desbalanceamento no número de imagens disponíveis para cada classe no dataset.

In [ ]:
classTotals = y_train.sum(axis=0)
classWeights = classTotals.max() / classTotals

classWeight = {
    i: classWeight
    for i, classWeight in enumerate(classWeights)
}

### Treinamento do modelo

Com o modelo definido, vamos agora compilá-lo e vamos também adicionar o optimizador Adam.

Depois disso, vamos treinar este modelo com base nos dados de treinamento que temos disponíveis.

In [ ]:
numEpochs = 30
learningRate = 1e-3

TrafficSignNet.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(lr=learningRate, decay=(learningRate/(0.5 * numEpochs))),
    metrics=["accuracy"]
)

In [ ]:
%%time

batchSize = 64

H = TrafficSignNet.fit_generator(
    imageGenerator.flow(X_train, y_train, batch_size=batchSize),
    validation_data=(X_val, y_val),
    steps_per_epoch=(len(X_train) // batchSize),
    epochs=numEpochs,
    class_weight=classWeight,
    verbose=True
)

In [ ]:
plt.plot(np.arange(numEpochs), H.history['accuracy'], label='train_acc')
plt.plot(np.arange(numEpochs), H.history['val_accuracy'], label='val_acc')
plt.title("Acurácia ao longo do treinamento.")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend(loc="lower right")
plt.show()

### Desempenho do modelo

Vamos primeiramente observar o desempenho do modelo com base nos dados de validação.

In [ ]:
predictions = TrafficSignNet.predict(X_val, batch_size=batchSize)

report = classification_report(
    np.asarray(y_val).argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=y_val.columns
)

print(report)

### Resultados para os dados de teste

Por fim, vamos realizar a classificação dos dados de teste e, então, vamos observar a qualidade da classificação feita.

In [ ]:
GTSRBTest = pd.read_csv(link + "Test.csv")
GTSRBTest.Path = link + GTSRBTest.Path

# União do dataset e das descrições
GTSRBTest = GTSRBTest.merge(GTSRBInfo, on="ClassId")

GTSRBTest.shape

In [ ]:
imagesTest = GTSRBTest.Path.map(plt.imread).map(imageTreatment)

In [ ]:
predictionsTest = TrafficSignNet.predict(np.stack(imagesTest), batch_size=batchSize)
y_test = pd.get_dummies(GTSRBTest.Label)

reportTest = classification_report(
    np.asarray(y_test).argmax(axis=1),
    predictionsTest.argmax(axis=1),
    target_names=y_test.columns
)

print(reportTest)

### Matriz de confusão

In [ ]:
plt.figure(figsize=(22, 12))

sns.heatmap(
    confusion_matrix(predictionsTest.argmax(axis=1), np.asarray(y_test).argmax(axis=1)),
    cmap=plt.cm.RdYlGn_r,
    xticklabels=y_test.columns,
    yticklabels=y_test.columns,
    annot=True,
    fmt='g'
)

### Visualização de classificações incorretas

In [ ]:
wrongPredicted = GTSRBTest.copy()
wrongPredicted.Label = wrongPredicted.Label.astype(str) + '\n' + "Res: " + y_test.columns[predictionsTest.argmax(axis=1)].astype(str)

plotTrafficSigns(
    wrongPredicted[predictionsTest.argmax(axis=1) != np.asarray(y_test).argmax(axis=1)],
    imRead=lambda classe: imagesTest[classe.name],
    random_state=random_state
)